<a href="https://colab.research.google.com/github/andresenrique08/ML_2020/blob/master/Reto%202/Reto2_ML_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reto 2

Autores:
*   Andrés Enrique Ávila Rojas (201319127)
*   Jeison Alexander Timana Rangel (201617870)

**Regresión Logística - Predicción de lluvia**

In [ ]:
# Import libraries 
import numpy as np 
import pandas as pd
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score # las dos medidas que se van a utilizar pero en diferentes datasets.
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


Se toman los datos del repositorio de github con el siguiente link https://github.com/andresenrique08/ML_2020/blob/master/Reto%202/weatherAUS.csv

In [ ]:
# Reading file from github
url = "https://raw.githubusercontent.com/andresenrique08/ML_2020/master/Reto%202/weatherAUS.csv"
data  = pd.read_csv(url)
data.head()

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RISK_MM,RainTomorrow
0,2008-12-01,Albury,13.4,22.9,0.6,NaN,NaN,W,44.0,W,WNW,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,NaN,16.9,21.8,No,0.0,No
1,2008-12-02,Albury,7.4,25.1,0.0,NaN,NaN,WNW,44.0,NNW,WSW,4.0,22.0,44.0,25.0,1010.6,1007.8,NaN,NaN,17.2,24.3,No,0.0,No
2,2008-12-03,Albury,12.9,25.7,0.0,NaN,NaN,WSW,46.0,W,WSW,19.0,26.0,38.0,30.0,1007.6,1008.7,NaN,2.0,21.0,23.2,No,0.0,No
3,2008-12-04,Albury,9.2,28.0,0.0,NaN,NaN,NE,24.0,SE,E,11.0,9.0,45.0,16.0,1017.6,1012.8,NaN,NaN,18.1,26.5,No,1.0,No
4,2008-12-05,Albury,17.5,32.3,1.0,NaN,NaN,W,41.0,ENE,NW,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,0.2,No


###Análisis de variables

Este análisis se realiza con el fin de conocer los datos con los que se generará el modelo y posteriormente análizarlos estadísticamente.

Se obtiene el tipo de cada una de las variables

In [ ]:
data.dtypes

Date              object
Location          object
MinTemp          float64
MaxTemp          float64
Rainfall         float64
Evaporation      float64
Sunshine         float64
WindGustDir       object
WindGustSpeed    float64
WindDir9am        object
WindDir3pm        object
WindSpeed9am     float64
WindSpeed3pm     float64
Humidity9am      float64
Humidity3pm      float64
Pressure9am      float64
Pressure3pm      float64
Cloud9am         float64
Cloud3pm         float64
Temp9am          float64
Temp3pm          float64
RainToday         object
RISK_MM          float64
RainTomorrow      object
dtype: object

In [ ]:
#Tamaño del dataframe
data.shape

(142193, 24)

Se determina si existen valores perdidos en cada una de las variables

In [ ]:
def missing_values_table(df): 
        val_per = df.isnull().sum()
        porcentaje_val_per = 100 * df.isnull().sum()/len(df)
        tabla_val_per = pd.concat([val_per, porcentaje_val_per], axis=1)
        tabla_val_per_ren_columnas = tabla_val_per.rename(
        columns = {0 : 'Valores perdidos', 1 : '% sobre el total'})
        return tabla_val_per_ren_columnas 

In [ ]:
missing_values_table(data)

,Valores perdidos,% sobre el total
Date,0,0.000000
Location,0,0.000000
MinTemp,637,0.447983
MaxTemp,322,0.226453
Rainfall,1406,0.988797
Evaporation,60843,42.789026
Sunshine,67816,47.692924
WindGustDir,9330,6.561504
WindGustSpeed,9270,6.519308
WindDir9am,10013,7.041838


Se verifica que no filas de datos repetidas

In [ ]:
data.duplicated().sum()

0

Identificación de los valores únicos de las variables RainToday y RainTomorrow

In [ ]:
data.RainToday.unique()

array(['No', 'Yes', nan], dtype=object)

In [ ]:
data.RainTomorrow.unique()

array(['No', 'Yes'], dtype=object)

In [ ]:
data.Location.unique()

array(['Albury', 'BadgerysCreek', 'Cobar', 'CoffsHarbour', 'Moree',
       'Newcastle', 'NorahHead', 'NorfolkIsland', 'Penrith', 'Richmond',
       'Sydney', 'SydneyAirport', 'WaggaWagga', 'Williamtown',
       'Wollongong', 'Canberra', 'Tuggeranong', 'MountGinini', 'Ballarat',
       'Bendigo', 'Sale', 'MelbourneAirport', 'Melbourne', 'Mildura',
       'Nhil', 'Portland', 'Watsonia', 'Dartmoor', 'Brisbane', 'Cairns',
       'GoldCoast', 'Townsville', 'Adelaide', 'MountGambier', 'Nuriootpa',
       'Woomera', 'Albany', 'Witchcliffe', 'PearceRAAF', 'PerthAirport',
       'Perth', 'SalmonGums', 'Walpole', 'Hobart', 'Launceston',
       'AliceSprings', 'Darwin', 'Katherine', 'Uluru'], dtype=object)

In [ ]:
data.WindGustDir.unique()

array(['W', 'WNW', 'WSW', 'NE', 'NNW', 'N', 'NNE', 'SW', 'ENE', 'SSE',
       'S', 'NW', 'SE', 'ESE', nan, 'E', 'SSW'], dtype=object)

###Estandarización de variables

Se crea un dataset con las variables no categoricas para realizar la estandarización y conocer la influencia de cada varible sobre la variable de interés, que en este caso es "RainTomorrow"

In [ ]:
temp_data = 

In [ ]:
data_estandar = StandardScaler()
data_estandar = pd.DataFrame(data_estandar.fit_transform(data), columns = data.columns) 
data_estandar.head()

ValueError: ignored